# Identifying Need for Change: North Carolina School Performance

Out of the 2,617 public schools (including Charter) operating in North Carolina during the 2016-2017 school year, 902 schools (34.5%) have, for at least one year since 2013-2014, been classified as a low performing school. 

NCPDI classifies low performing schools as:

“Low-performing schools are those that receive a **school performance grade** of **D** or **F** and a **school growth score** of **"met expected growth"** or **"not met expected growth"** as defined by G.S. 115C-83.15.” (G.S. 115C-105.37(a)), and

“A Low-performing local school administrative unit is a unit in which the majority of the schools in that unit that received a school performance grade and school growth score as provided in G.S. 115C-83.15 have been identified as low-performing schools, as provided in G.S. 115C-105.37.” (G.S. 115C-105.39A(a)).

Source: http://www.ncpublicschools.org/schooltransformation/low-performing/

**Thus, we treat low performing schools as a proxy for aggregate student educational achievement.**


## Problem Statement: 
In recent years, 30% of public schools in North Carolina have been low performing. Students in low performing schools are not meeting the educational achievement standards set by the state. Factors outside administrators' control: economically disadvantaged and majority-minority student populations, are the most influential indicators of low performance.  


## Motivation: 
Of the 902 schools low performing between 2013/14 and 2016/17, 209 have been low performing for all schools years (8%), 203 have been low performing for 3 of the schools years (7.8%), 227 have been low performing for 2 schools years (8.7%), and 263 have been low performing once (10%). Another way to look at these numbers is to consider that out of the four school years between 2013/14 and 2016/2017, 24.4% of schools have been recurringly low performing. 

What does this mean in terms of students? 
Out of around the 1.5 million total number of students studying in a public school in the 2016/17 school year, around 460,000 of those students (30%) have had at least a year studying a low performing school. Around 100,000 students (6.8%) in North Carolina study at a school that has been low performing for 4 years. 


## Solution:
We identify the factors within school administrator's control that can positively impact a school's EVAAS growth score. Growth scores are a metric to measure how well a school's performance increases over a year. We focus on growth instead of raw performance scores, as raw scores will be much slower to change over time. 

This notebook reviews NCPDI North Carolina School Report Card and Statistical Profile data to identify the factors that contribute to low school performance and EVAAS growth scores. We will then create a model to predict EVAAS growth scores including only school-level factors irrespective of student demographics. After determining the most predictive factors for determining EVAAS growth scores, we simulate changes in these factors to demonstrate a theoretical improvement in student achievement growth. 

We: 
1. Take a look at the heuristically most common reasons for low school performance: percentage of economically disadvantaged students, student demographics, and school funding to see if these indicators are statistically different in low performing schools. 

2. Remove the factors outside of the school adminsitration's control from the dataset to determine which school-level factors are most important in determining low performance by:
    1. Performing Feature Importance using XGBoost. XGBoost is a tree-based gradient-boosting method which minimizes a cost function relative to predicting a target variable. When a node in a decision tree is split, we can calculate the following reduction in impurity, and attribute this reduction to feature involved. When the tree is finished splitting nodes, those features with the largest proportional contribution toward decreasing impurity within nodes can be said to be the most “important.”
    
4. **Test the predictive nature of our selected features by creating a classification model to predict the EVAAS growth score for each of our 4 years. Whether these features are important will be reflected in the accuracy and precision of the regression model. **

5. Use a new methodology for decision-making. We create a function that will synthesize data based on percent changes in a certain input feature to then be used in our regression model to predict low performance. This will allow us to review, all things being equal, how a change in one or more features may correlate to a change in school EVAAS growth score. 


*Please note: Dataset Creation and Processing can be found: https://github.com/oleeson/NCPDI-Capstone*

In [1]:
#import required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

import seaborn as sns
sns.set(color_codes=True)

import warnings
warnings.filterwarnings("ignore")

#Change Dir to Import Dataset for EDA 
os.chdir("..")
cwd = os.getcwd()
cwd = cwd + "/DatasetCreation"

In [2]:
## Read in Schools Dataset
df_14 = pd.read_csv(cwd+'/2014/PublicSchools2014_LPS_Processed.csv')
df_15 = pd.read_csv(cwd+'/2015/PublicSchools2015_LPS_Processed.csv')
df_16 = pd.read_csv(cwd+'/2016/PublicSchools2016_LPS_Processed.csv')
df_17 = pd.read_csv(cwd+'/2017/PublicSchools2017_LPS_Processed.csv')

In [3]:
#Create Column Subsets
profileCols = ['title1_type_cd',
 'clp_ind',
 'focus_clp_ind',
 'summer_program_ind',
 'asm_no_spg_ind',
 'no_data_spg_ind',
 'student_num',
 'lea_avg_student_num',
 'Grad_project_status', 
 'category_cd_E',
 'category_cd_H',
 'category_cd_I',
 'category_cd_M',
 'category_cd_T',
 'esea_status_Esea_Pass',
 'esea_status_Non_Esea',
 'SBE District_Northeast',
 'SBE District_Northwest',
 'SBE District_Piedmont-Triad',
 'SBE District_Sandhills',
 'SBE District_Southeast',
 'SBE District_Southwest',
 'SBE District_Western']

demographicCols = ['AsianFemalePct',
 'AsianMalePct',
 'AsianPct',
 'BlackFemalePct',
 'BlackMalePct',
 'BlackPct',
 'HispanicFemalePct',
 'HispanicMalePct',
 'HispanicPct',
 'IndianFemalePct',
 'IndianMalePct',
 'IndianPct',
 'MinorityFemalePct',
 'MinorityMalePct',
 'MinorityPct',
 'PacificIslandFemalePct',
 'PacificIslandMalePct',
 'PacificIslandPct',
 'TwoOrMoreFemalePct',
 'TwoOrMoreMalePct',
 'TwoOrMorePct',
 'WhiteFemalePct',
 'WhiteMalePct',
 'WhitePct',
 'pct_eds']

environmentCols = ['avg_daily_attend_pct',
 'crime_per_c_num',
 'short_susp_per_c_num',
 'long_susp_per_c_num',
 'expelled_per_c_num',
 'stud_internet_comp_num',
 'lea_avg_daily_attend_pct',
 'lea_crime_per_c_num',
 'lea_short_susp_per_c_num',
 'lea_long_susp_per_c_num',
 'lea_expelled_per_c_num',
 'lea_stud_internet_comp_num',
 'digital_media_pct',
 'avg_age_media_collection',
 'books_per_student',
 'lea_avg_age_media_collection',
 'lea_books_per_student',
 'wap_num',
 'wap_per_classroom',
 'lea_wap_num',
 'lea_wap_per_classroom',
 'Byod_Yes',
 '_1_to_1_access_Yes',
 'SRC_devices_sent_home_Yes']

educatorCols = ['flicensed_teach_pct',
 'tchyrs_0thru3_pct',
 'tchyrs_4thru10_pct',
 'tchyrs_11plus_pct',
 'class_teach_num',
 'nbpts_num',
 'advance_dgr_pct',
 '_1yr_tchr_trnovr_pct',
 'lea_flicensed_teach_pct',
 'lea_tchyrs_0thru3_pct',
 'lea_tchyrs_4thru10_pct',
 'lea_tchyrs_11plus_pct',
 'lea_class_teach_num',
 'lea_nbpts_num',
 'lea_advance_dgr_pct',
 'lea_1yr_tchr_trnovr_pct',
 'lea_lateral_teach_pct',
 '0-3 Years_Exp_Pct_Tch',
 '10+ Years_Exp_Pct_Tch',
 '4-10 Years_Exp_Pct_Tch',
 '0-3 Years_LEA_Exp_Pct_Prin',
 '10+ Years_LEA_Exp_Pct_Prin',
 '4-10 Years_LEA_Exp_Pct_Prin',
 'Accomplished_TCHR_Standard 1_Pct',
 'Accomplished_TCHR_Standard 2_Pct',
 'Accomplished_TCHR_Standard 3_Pct',
 'Accomplished_TCHR_Standard 4_Pct',
 'Accomplished_TCHR_Standard 5_Pct',
 'Developing_TCHR_Standard 1_Pct',
 'Developing_TCHR_Standard 2_Pct',
 'Developing_TCHR_Standard 3_Pct',
 'Developing_TCHR_Standard 4_Pct',
 'Developing_TCHR_Standard 5_Pct',
 'Distinguished_TCHR_Standard 1_Pct',
 'Distinguished_TCHR_Standard 2_Pct',
 'Distinguished_TCHR_Standard 3_Pct',
 'Distinguished_TCHR_Standard 4_Pct',
 'Distinguished_TCHR_Standard 5_Pct',
 'Does Not Meet Expected Growth_TCHR_Student Growth_Pct',
 'Exceeds Expected Growth_TCHR_Student Growth_Pct',
 'Meets Expected Growth_TCHR_Student Growth_Pct',
 'Not Demostrated_TCHR_Standard 1_Pct',
 'Not Demostrated_TCHR_Standard 2_Pct',
 'Not Demostrated_TCHR_Standard 3_Pct',
 'Not Demostrated_TCHR_Standard 4_Pct',
 'Not Demostrated_TCHR_Standard 5_Pct',
 'Proficient_TCHR_Standard 1_Pct',
 'Proficient_TCHR_Standard 2_Pct',
 'Proficient_TCHR_Standard 3_Pct',
 'Proficient_TCHR_Standard 4_Pct',
 'Proficient_TCHR_Standard 5_Pct']

fundingCols = ['lea_total_expense_num',
 'lea_salary_expense_pct',
 'lea_services_expense_pct',
 'lea_supplies_expense_pct',
 'lea_instruct_equip_exp_pct',
 'lea_federal_perpupil_num',
 'lea_local_perpupil_num',
 'lea_state_perpupil_num']

performCols = ['SPG Score',
 'EVAAS Growth Score',
 'Overall Achievement Score',
 'TotalTargets_pTarget_PctMet',
 'lea_sat_avg_score_num',
 'lea_sat_participation_pct',
 'lea_ap_participation_pct',
 'lea_ap_pct_3_or_above',
 'LPS_14',
 'LPS_15',
 'LPS_16',
 'LPS_17',
 'RLPS',
 'SPG Grade_A+NG',
 'SPG Grade_B',
 'SPG Grade_C',
 'SPG Grade_D',
 'SPG Grade_F',
 'SPG Grade_I',
 'Reading SPG Grade_B',
 'Reading SPG Grade_C',
 'Reading SPG Grade_D',
 'Reading SPG Grade_F',
 'Math SPG Grade_B',
 'Math SPG Grade_C',
 'Math SPG Grade_D',
 'Math SPG Grade_F',
 'EVAAS Growth Status_Met',
 'EVAAS Growth Status_NotMet',
 'State Gap Compared_Y']

featureImportance = ['student_num', 'class_teach_num', 'lea_class_teach_num', 
                     'wap_num', 'wap_per_classroom', 'stud_internet_comp_num', 
                     'lea_stud_internet_comp_num', 'summer_program_ind',
                     'avg_daily_attend_pct', 'lea_avg_daily_attend_pct',
                     'short_susp_per_c_num', 'lea_short_susp_per_c_num', 
                     'lea_crime_per_c_num', 'crime_per_c_num','flicensed_teach_pct', 
                     'nbpts_num', 'tchyrs_0thru3_pct', 'tchyrs_4thru10_pct', 
                     'lea_tchyrs_4thru10_pct', 'advance_dgr_pct', 'lea_advance_dgr_pct',
                     'Accomplished_TCHR_Standard 1_Pct', '10+ Years_LEA_Exp_Pct_Prin'] 

In [4]:
combined_cols = (environmentCols + fundingCols + educatorCols + profileCols)

In [5]:
df_sub_17 = df_17[df_17.columns.intersection(combined_cols)]
df_imp_17 = df_17[df_17.columns.intersection(featureImportance)]

In [6]:
# create x explanatory and y response variables for regression
GS_17 = df_17['EVAAS Growth Score']
unit_code = df_17['unit_code']

X_combined = df_sub_17
X_important = df_imp_17

Y = GS_17

#inspect data 
print('combined: \n')
print(X_combined.info(), '\n')
print('important: \n')
print(X_important.info())

combined: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2443 entries, 0 to 2442
Columns: 106 entries, title1_type_cd to SRC_devices_sent_home_Yes
dtypes: float64(82), int64(24)
memory usage: 2.0 MB
None 

important: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2443 entries, 0 to 2442
Data columns (total 23 columns):
summer_program_ind                  2443 non-null int64
student_num                         2443 non-null float64
avg_daily_attend_pct                2443 non-null float64
crime_per_c_num                     2443 non-null float64
short_susp_per_c_num                2443 non-null float64
stud_internet_comp_num              2443 non-null float64
lea_avg_daily_attend_pct            2443 non-null float64
lea_crime_per_c_num                 2443 non-null float64
lea_short_susp_per_c_num            2443 non-null float64
lea_stud_internet_comp_num          2443 non-null float64
wap_num                             2443 non-null float64
wap_per_classroom                

## Cross Validation
**Cross validation is performed using repeated holdout using ShuffleSplit()**
* Ten folds are used
* The split is: 90% training data and 10% test data
* A random seed is set so the same random test and training splits are used each time cross validation is performed.

In [7]:
#Divide data into test and training splits
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.10, random_state=0)

## Custom Scorers for Evaluating Regression Models 

**All regression models created in this notebook are validated using the following metrics:**
* Mean Absolute Error (MAE)
* Root Mean Squared Error (RMSE) - https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python

**For details on making scorers to return multiple mean error scores see:**
* http://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html
* https://github.com/scikit-learn/scikit-learn/pull/7388
* https://github.com/drorata/multiscorer

In [8]:
#Use mean absolute error (MAE) to score the regression models created 
#(the scale of MAE is identical to the response variable)
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error

#Function for Root mean squared error
#https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))


#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'RMSE': rmse_scorer,
               } 

## Regression Model Evaluation
** All regression models are evaluated using the regression model evaluation function below: ** 
* The following regression evaluation function uses the cross validation object and the custom scorers in the two cells above in combination with sklearn.model_selection's cross_validate function to perform cross validation for regression estimators.
* The cross validation object above uses a random seed to ensure that all regression estimators are tested on the same randomly selected records for each cross validation fold.
* Custom scorers are created using the three chosen mean error scores and passed into cross_validate(), so all three scores are calcualted using a single call to cross_validate().
* All of this functionality is wrapped within the custom EvaluateRegressionEstimator() function below so multiple regression models may be tested using the same test / train cv data and evaluation scores producing a consistent output for each model without the need to re-write the same code over and over. 

In [9]:
from sklearn.model_selection import cross_validate

def EvaluateRegressionEstimator(regEstimator, X, y, cv):
    
    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))


    #print mean MAE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))
    print('*********************************************************')

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['RMSE'] = scores['test_RMSE']
    return scoresResults


## SVM Regression

In [10]:
from sklearn.svm import SVR

#Create a regression estimator with best parameters for cross validation
regEstimator = SVR(C=0.01, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                   kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
EvaluateRegressionEstimator(regEstimator, X_combined, Y, cv)

The average MAE for all cv folds is: 			 13.071
The average RMSE for all cv folds is: 			 34.72
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,RMSE
0,11.362843,17.424371
1,12.052729,19.459643
2,11.765161,18.740860
3,17.026864,86.550308
4,11.665364,19.737275
5,11.981631,19.125772
6,15.940998,66.841656
7,12.797977,20.560340
8,14.809147,60.163679
9,11.306857,18.593574
